In [31]:
import sys,os
[sys.path.append(x[0]) for x in os.walk('SL2P_PYTHON_package')]
import dictionariesSL2P
import SL2P
from matplotlib import pyplot as plt
import os,numpy
import rasterio
import os
import pandas as pd

### Initiation

In [83]:
variableName='fAPAR'
imageCollectionName="S2_L2A"
fn='./testdata/Surface_refelctance_S2_L2A.csv'

#### Input data Control ***-
if variableName not in dictionariesSL2P.make_net_options().keys():
    raise ValueError(('Supported vegetation valiables: %s'%(dictionariesSL2P.make_net_options().keys())))
if imageCollectionName not in dictionariesSL2P.make_net_options()[variableName].keys():
    raise ValueError(('Supported EO datasets: %s'%(dictionariesSL2P.make_net_options()[variableName].keys())))  
       
sl2p_inputs=dictionariesSL2P.make_net_options()[variableName][imageCollectionName]['inputBands']  
print ('SL2P input data format for %s data: %s' %(imageCollectionName,sl2p_inputs))

SL2P input data format for S2_L2A data: ['cosVZA', 'cosSZA', 'cosRAA', 'B03', 'B04', 'B05', 'B06', 'B07', 'B8A', 'B11', 'B12']


In [59]:
def prepare_SL2P_inp(inp):
    inp=inp[sl2p_inputs] 
    inp=inp.values.T
    inp=numpy.reshape(inp,(inp.shape[0],inp.shape[1],-1))
    return inp

In [60]:
def run_SL2P(inp,variableName,imageCollectionName):
    varmap=SL2P.SL2P(inp,variableName,imageCollectionName)
    return varmap  

### 1- Read / prepare data

In [67]:
data=pd.read_csv(fn)

In [ ]:
inp=prepare_SL2P_inp(data)

### 2 - Run SL2P / export outputs

In [69]:
varmap=run_SL2P(inp,variableName,imageCollectionName)

Estimating fAPAR from S2_L2A data
Generating sl2p input data flag
Run SL2P...
SL2P start: 2024-02-26 13:29:51.035962
SL2P end: 2024-02-26 13:29:51.055842
Generating sl2p output product flag
Done


### 3- Export outputs

In [72]:
df=inp[['date', 'longitude', 'latitude','QC']]
for kk in list(varmap.keys()):
    df.loc[:,kk]=varmap[kk].T[0]  
df.to_csv(fn.replace('.csv','_%s.csv'%(variableName)))    